This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/2023.2.1_prerelease/wallaroo-features/pipeline_multiple_replicas_forecast_tutorial).

## Statsmodel Forecast with Wallaroo Features: Model Creation

This tutorial series demonstrates how to use Wallaroo to create a Statsmodel forecasting model based on bike rentals.  This tutorial series is broken down into the following:

* Create and Train the Model:  This first notebook shows how the model is trained from existing data.
* Deploy and Sample Inference:  With the model developed, we will deploy it into Wallaroo and perform a sample inference.
* Parallel Infer:  A sample of multiple weeks of data will be retrieved and submitted as an asynchronous parallel inference.  The results will be collected and uploaded to a sample database.
* External Connection:  A sample data connection to Google BigQuery to retrieve input data and store the results in a table.
* ML Workload Orchestration:  Take all of the previous steps and automate the request into a single Wallaroo ML Workload Orchestration.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

In [3]:
import pandas as pd
import datetime
import os

from statsmodels.tsa.arima.model import ARIMA

from resources import simdb as simdb

import importlib
importlib.reload(simdb)

<module 'resources.simdb' from '/Users/jhansarick/Storage/github/WallarooLabs/Wallaroo_Tutorials/wallaroo-features/pipeline_multiple_replicas_forecast_tutorial/resources/simdb.py'>

### Train the Model

The resources to train the model will start with the local file `./data/test_data.csv`.  This data is load and prepared for use in training the model.

For this example, the simulated database is controlled by the resources `simbdb`.

In [5]:
def mk_dt_range_query(*, tablename: str, seed_day: str) -> str:
    assert isinstance(tablename, str)
    assert isinstance(seed_day, str)
    query = f"select count from {tablename} where date > DATE(DATE('{seed_day}'), '-1 month') AND date <= DATE('{seed_day}')"
    return query

conn = simdb.get_db_connection()

# create the query
query = mk_dt_range_query(tablename=simdb.tablename, seed_day='2011-03-01')
print(query)

# read in the data
training_frame = pd.read_sql_query(query, conn)

# convert into a total dataframe

evaluation_dataframe = pd.DataFrame({'count': [training_frame['count'].values.tolist()]})
evaluation_dataframe

select count from bikerentals where date > DATE(DATE('2011-03-01'), '-1 month') AND date <= DATE('2011-03-01')


,count
0,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 160..."


## Test the Forecast

The training frame is then loaded, and tested against our `forecast` model.

In [6]:
# test
from models import forecast_standard
import json

forecast_standard.wallaroo_json(evaluation_dataframe)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740],
  'weekly_average': [1745.2857142857142]}]

### Reload New Model

The `forecast` model is reloaded in preparation of creating the evaluation data.

In [4]:
import importlib
importlib.reload(forecast)

<module 'forecast' from '/home/jovyan/pipeline_multiple_replicas_forecast_tutorial/forecast.py'>

### Prepare evaluation data

For ease of inference, we save off the evaluation data to a separate json file.

In [7]:
# save off the evaluation frame json, too

evaluation_dataframe.to_json('./data/testdata_dict.df.json', orient="records")
